In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

#tfe.enable_eager_execution()


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_Mark||',
        '?': '||Question_Mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parenthese||',
        '--': '||Dash||',
        '\n': '||Return||'
    }
    return token_dict

In [5]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.float32, shape=[None, None], name="input")
    targets = tf.placeholder(tf.float32, shape=[None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    return input, targets, learning_rate

In [6]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [7]:
from tensorflow.contrib import rnn

In [8]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    
    embedding = tf.get_variable("embedding", [vocab_size, embed_dim])
    output = tf.nn.embedding_lookup(embedding, input_data)
    return output

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    def build_cell(rnn_size):
        basic_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
        return basic_cell
        
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size) for _ in range(2)])
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name="initial_state")
    
    return cell, initial_state

def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, "final_state")
    return outputs, final_state

In [10]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    #stacked_outputs = tf.reshape(outputs, [-1, rnn_size])
    logits = tf.layers.dense(outputs, vocab_size)
    
    return logits, final_state

In [16]:
test_input_data_shape = [128, 5]
test_input_data = tf.placeholder(tf.int32, test_input_data_shape)
test_rnn_size = 256
test_embed_dim = 300
test_rnn_layer_size = 2
test_vocab_size = 27    

In [17]:
test_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(test_rnn_size) 
                             for _ in range(test_rnn_layer_size)])

In [18]:
outputs, final_state = build_nn(test_cell, 
                                test_rnn_size,
                                test_input_data,
                                test_vocab_size,
                                test_embed_dim)

ValueError: Variable embedding already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-8-69481447b453>", line 11, in get_embed
    embedding = tf.get_variable("embedding", [vocab_size, embed_dim])
  File "<ipython-input-10-252466c9a36f>", line 12, in build_nn
    embed = get_embed(input_data, vocab_size, embed_dim)
  File "<ipython-input-13-c78c2b17fec8>", line 5, in <module>
    test_embed_dim)


In [19]:
outputs

<tf.Tensor 'dense/BiasAdd:0' shape=(?, 5, 27) dtype=float32>

In [20]:
final_state

<tf.Tensor 'final_state:0' shape=(2, 2, ?, 256) dtype=float32>